In [33]:
import sys
import pandas as pd
import numpy as np
data = pd.read_csv('data/train.csv', encoding = 'big5') # 读取文件
data

,日期,測站,測項,0,1,2,3,4,5,6,...,14,15,16,17,18,19,20,21,22,23
0,2014/1/1,豐原,AMB_TEMP,14,14,14,13,12,12,12,...,22,22,21,19,17,16,15,15,15,15
1,2014/1/1,豐原,CH4,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,2014/1/1,豐原,CO,0.51,0.41,0.39,0.37,0.35,0.3,0.37,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,2014/1/1,豐原,NMHC,0.2,0.15,0.13,0.12,0.11,0.06,0.1,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,2014/1/1,豐原,NO,0.9,0.6,0.5,1.7,1.8,1.5,1.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,2014/12/20,豐原,THC,1.8,1.8,1.8,1.8,1.8,1.7,1.7,...,1.8,1.8,2,2.1,2,1.9,1.9,1.9,2,2
4316,2014/12/20,豐原,WD_HR,46,13,61,44,55,68,66,...,59,308,327,21,100,109,108,114,108,109
4317,2014/12/20,豐原,WIND_DIREC,36,55,72,327,74,52,59,...,18,311,52,54,121,97,107,118,100,105
4318,2014/12/20,豐原,WIND_SPEED,1.9,2.4,1.9,2.8,2.3,1.9,2.1,...,2.3,2.6,1.3,1,1.5,1,1.7,1.5,2,2


In [34]:
#取需要的数值部分將 'RAINFALL' 欄位全部補 0
data = data.iloc[:, 3:]
data[data == 'NR'] = 0
raw_data = data.to_numpy()
raw_data

H:\anaconda\envs\pytorch\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
H:\anaconda\envs\pytorch\lib\site-packages\pandas\core\frame.py:3086: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


array([['14', '14', '14', ..., '15', '15', '15'],
       ['1.8', '1.8', '1.8', ..., '1.8', '1.8', '1.8'],
       ['0.51', '0.41', '0.39', ..., '0.35', '0.36', '0.32'],
       ...,
       ['36', '55', '72', ..., '118', '100', '105'],
       ['1.9', '2.4', '1.9', ..., '1.5', '2', '2'],
       ['0.7', '0.8', '1.8', ..., '1.6', '1.8', '2']], dtype=object)

In [63]:
month_data = {}
for month in range(12):
    sample = np.empty([18, 480])
    for day in range(20):
        sample[:, day * 24 : (day+1)*24] = raw_data[18*(20*month+day) : 18*(20*month + day+1), :]
    month_data[month] = sample
month_data[0].T

array([[14.  ,  1.8 ,  0.51, ..., 35.  ,  1.4 ,  0.5 ],
       [14.  ,  1.8 ,  0.41, ..., 79.  ,  1.8 ,  0.9 ],
       [14.  ,  1.8 ,  0.39, ...,  2.4 ,  1.  ,  0.6 ],
       ...,
       [14.  ,  1.8 ,  0.34, ..., 48.  ,  1.1 ,  1.2 ],
       [13.  ,  1.8 ,  0.41, ..., 63.  ,  1.9 ,  1.2 ],
       [13.  ,  1.8 ,  0.43, ..., 53.  ,  1.9 ,  1.3 ]])

In [65]:
x = np.empty([12 * 471, 18 * 9], dtype = float)
y = np.empty([12 * 471, 1], dtype = float)
for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day == 19 and hour > 14:
                continue
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) #vector dim:18*9 (9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9)
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value
print(x)
print(y)
x.shape

[[14.  14.  14.  ...  2.   2.   0.5]
 [14.  14.  13.  ...  2.   0.5  0.3]
 [14.  13.  12.  ...  0.5  0.3  0.8]
 ...
 [17.  18.  19.  ...  1.1  1.4  1.3]
 [18.  19.  18.  ...  1.4  1.3  1.6]
 [19.  18.  17.  ...  1.3  1.6  1.8]]
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]


(5652, 162)

In [68]:
mean_x = np.mean(x, axis = 0) #18 * 9 
std_x = np.std(x, axis = 0) #18 * 9 
for i in range(len(x)): #12 * 471
    for j in range(len(x[0])): #18 * 9 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j]
mean_x.shape

(162,)